<a href="https://colab.research.google.com/github/yuto-sb/my-repository/blob/main/GCI%E3%82%B3%E3%83%B3%E3%83%9A%E3%82%A2%E3%83%B3%E3%82%B5%E3%83%B3%E3%83%96%E3%83%AB%E6%B8%88%E3%81%BF%E3%81%9D%E3%81%AE3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

import warnings
warnings.filterwarnings('ignore')

In [ ]:
path =  '/content/drive/MyDrive/02.コンペ1（10 28公開済）/'

df = pd.read_csv(path + 'data/train.csv')
df_test = pd.read_csv(path + 'data/test.csv')

In [ ]:
df.head(10)

,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,1,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,1,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,1,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,1,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,0,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

# ==========================================
# 1. 基本的な欠損処理と削除
# ==========================================
# Cabinは削除
#df.drop('Cabin', axis=1, inplace=True)
#df_test.drop('Cabin', axis=1, inplace=True)

# Embarkedの欠損埋め
df['Embarked'].fillna('S', inplace=True)
df_test['Embarked'].fillna('S', inplace=True)

# Fareの欠損埋め (Testデータのみ)
fare_median = df_test.loc[(df_test['Embarked'] == 'S') & (df_test['Pclass'] == 3), 'Fare'].median()
df_test['Fare'].fillna(fare_median, inplace=True)

# Sexの数値化
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df_test['Sex'] = df_test['Sex'].replace({'male': 0, 'female': 1})


# ==========================================
# 2. TicketGroupの作成 (ここを先にやる！)
# ==========================================
# 全データでチケットの出現回数を数える
all_ticket_counts = pd.concat([df['Ticket'], df_test['Ticket']]).value_counts().to_dict()

df['TicketGroup'] = df['Ticket'].map(all_ticket_counts)
df_test['TicketGroup'] = df_test['Ticket'].map(all_ticket_counts)

# Ticket_labelの作成
for data in [df, df_test]:
    data['Ticket_label'] = 0
    data.loc[(data['TicketGroup'] >= 2) & (data['TicketGroup'] <= 4), 'Ticket_label'] = 2
    data.loc[(data['TicketGroup'] >= 5) & (data['TicketGroup'] <= 8) | (data['TicketGroup'] == 1), 'Ticket_label'] = 1
    data.loc[(data['TicketGroup'] >= 11), 'Ticket_label'] = 0


# ==========================================
# 3. Ageの欠損埋め (RandomForest)
# ==========================================
# 学習用とテスト用を結合
df_all = pd.concat([df, df_test], sort=False)
features = ['Pclass', 'Sex', 'SibSp', 'Parch'] # 必要に応じてTicketGroupを足してもOK

# Ageがあるデータで学習
known_age_all = df_all[df_all['Age'].notnull()]
rf_model = RandomForestRegressor(random_state=0, n_estimators=100, n_jobs=-1)
rf_model.fit(known_age_all[features], known_age_all['Age'])

# 予測と代入
# df
unknown_age_df = df[df['Age'].isnull()]
if len(unknown_age_df) > 0:
    df.loc[df['Age'].isnull(), 'Age'] = rf_model.predict(unknown_age_df[features])

# df_test
unknown_age_test = df_test[df_test['Age'].isnull()]
if len(unknown_age_test) > 0:
    df_test.loc[df_test['Age'].isnull(), 'Age'] = rf_model.predict(unknown_age_test[features])


# ==========================================
# 4. その他の特徴量エンジニアリング
# ==========================================

# --- Fare_Per_Person (TicketGroupができたので今なら計算できる) ---
df['Fare_Per_Person'] = df['Fare'] / df['TicketGroup']
df_test['Fare_Per_Person'] = df_test['Fare'] / df_test['TicketGroup']
df['Fare_Per_Person'] = df['Fare_Per_Person'].astype(int)
df_test['Fare_Per_Person'] = df_test['Fare_Per_Person'].astype(int)

# --- Family (SibSp + Parch) ---
for data in [df, df_test]:
    data['Family'] = data['SibSp'] + data['Parch'] + 1
    data.loc[(data['Family'] >= 2) & (data['Family'] <= 4), 'Family_label'] = 2
    data.loc[(data['Family'] >= 5) & (data['Family'] <= 7) | (data['Family'] == 1), 'Family_label'] = 1
    data.loc[(data['Family'] >= 8), 'Family_label'] = 0

# --- Title (敬称) ---
for data in [df, df_test]:
    data['Title'] = data['Name'].map(lambda x: x.split(', ')[1].split('. ')[0])
    data['Title'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer', inplace=True)
    data['Title'].replace(['Don', 'Sir',  'the Countess', 'Lady', 'Dona'], 'Royalty', inplace=True)
    data['Title'].replace(['Mme', 'Ms'], 'Mrs', inplace=True)
    data['Title'].replace(['Mlle'], 'Miss', inplace=True)
    data['Title'].replace(['Jonkheer'], 'Master', inplace=True)

# --- Surname & FamilyGroup (WCG用) ---
for data in [df, df_test]:
    data['Surname'] = data['Name'].map(lambda name:name.split(',')[0].strip())
    data['FamilyGroup'] = data['Surname'].map(data['Surname'].value_counts())

# ==========================================
# 5. WCG (Woman-Child-Group) ドーピング
# ==========================================
# 生存率の算出
Female_Child_Group = df.loc[(df['FamilyGroup']>=2) & ((df['Age']<=16) | (df['Sex']==1))]
Female_Child_Group = Female_Child_Group.groupby('Surname')['Perished'].mean()
Male_Adult_Group = df.loc[(df['FamilyGroup']>=2) & (df['Age']>16) & (df['Sex']==0)]
Male_Adult_List = Male_Adult_Group.groupby('Surname')['Perished'].mean()

# リスト作成
Dead_list = set(Female_Child_Group[Female_Child_Group.apply(lambda x:x==1)].index)
Survived_list = set(Male_Adult_List[Male_Adult_List.apply(lambda x:x==0)].index)

for data in [df, df_test]:
    # 条件1: テストデータかどうか（Perishedがない、またはPerishedが欠損している）
    if 'Perished' in data.columns:
        mask = data['Perished'].isnull() # Trainデータなら空欄の行だけ対象
    else:
        # Testデータなら全行が対象なので、全部Trueの列を作る
        mask = pd.Series(True, index=data.index)

    # 条件2: 名前リストに入っているか
    is_in_dead = data['Surname'].apply(lambda x: x in Dead_list)
    is_in_survived = data['Surname'].apply(lambda x: x in Survived_list)

    # データの書き換え
    # Sexはすでに0, 1に変換済みなので、代入する値も 0(male), 1(female) にします

    # Dead List -> Male(0), 28, Mr
    data.loc[mask & is_in_dead, ['Sex','Age','Title']] = [0, 28.0, 'Mr']

    # Survived List -> Female(1), 5, Mrs
    data.loc[mask & is_in_survived, ['Sex','Age','Title']] = [1, 5.0, 'Mrs']

print("WCG修正完了")

# ==========================================
# ★ 追加：泥臭い特徴量（過学習対策・圧縮版）
# ==========================================

# 1. Deck (Cabin) -> シンプルに「持っているかどうか」だけにする
# 細かい A, B, C... は Pclass と情報がかぶるため、過学習の原因になりやすいです。
# 「Cabinデータがある＝富裕層や特定の運用」という事実だけを抽出します。
for data in [df, df_test]:
    # Cabinが欠損していなければ 1, 欠損なら 0
    data['Has_Cabin'] = data['Cabin'].apply(lambda x: 0 if pd.isnull(x) else 1)

# 2. Ticket Prefix -> 「よく出るやつ」以外は 'Other' にまとめる
def clean_ticket(ticket):
    ticket = ticket.replace('.', '').replace('/', '').split()
    ticket = map(lambda t: t.strip(), ticket)
    ticket = list(filter(lambda t: not t.isdigit(), ticket))
    if len(ticket) > 0:
        return ticket[0]
    else:
        return 'Num' # 数字のみ

# まずは全データで抽出
all_prefixes = pd.concat([df['Ticket'], df_test['Ticket']]).map(clean_ticket)

# 出現回数をカウント
prefix_counts = all_prefixes.value_counts()

# 「10回以上出てくるメジャーなもの」だけリスト化
major_prefixes = prefix_counts[prefix_counts >= 10].index

# マッピング処理
for data in [df, df_test]:
    # 一旦全部抽出
    temp_prefix = data['Ticket'].map(clean_ticket)
    # メジャーなもの以外は 'Other' に置換（ここが過学習対策の肝！）
    data['Ticket_Prefix'] = temp_prefix.apply(lambda x: x if x in major_prefixes else 'Other')

# 3. Name Length (名前の長さ)
# これは数値情報なので、過学習しにくいです。そのまま使います。
for data in [df, df_test]:
    data['Name_Length'] = data['Name'].apply(len)
    # ただし、長すぎる外れ値をクリップしておくとより安全です
    data['Name_Length'] = data['Name_Length'].clip(upper=50)

# ==========================================
# 6. Binning (階級化) と Encoding
# ==========================================
# Age/Fare Code
for data in [df, df_test]:
    data['Age_Code'] = pd.cut(data['Age'], 5, labels=False)
    data['Fare_Code'] = pd.qcut(data['Fare'], 4, labels=False)

# ==========================================
# 列の選定とエンコーディング
# ==========================================
# Deckの代わりに Has_Cabin を入れます
target_cols = [
    'Perished','Pclass','Sex','Age','Fare','Fare_Per_Person',
    'Fare_Code', 'Age_Code', 'Embarked','Title','Family_label',
    'Ticket_label', 'Has_Cabin', 'Ticket_Prefix', 'Name_Length'
]

test_cols = [
    'Pclass','Sex','Age','Fare','Fare_Per_Person',
    'Fare_Code', 'Age_Code', 'Embarked','Title','Family_label',
    'Ticket_label', 'Has_Cabin', 'Ticket_Prefix', 'Name_Length'
]

df = df[target_cols]
df_test = df_test[test_cols]

# One-Hot Encoding
# Ticket_Prefix の種類が激減しているので、列数は爆発しません
df = pd.get_dummies(df, columns=[
    'Pclass', 'Sex', 'Embarked', 'Title', 'Family_label',
    'Ticket_label', 'Age_Code', 'Fare_Code',
    'Has_Cabin', 'Ticket_Prefix'
])

df_test = pd.get_dummies(df_test, columns=[
    'Pclass', 'Sex', 'Embarked', 'Title', 'Family_label',
    'Ticket_label', 'Age_Code', 'Fare_Code',
    'Has_Cabin', 'Ticket_Prefix'
])
# 列合わせ (Align)
df, df_test = df.align(df_test, join='left', axis=1, fill_value=0)

# NumPy配列化
y = df['Perished'].values
X = df.drop('Perished', axis=1).values
X_test = df_test.drop('Perished', axis=1).values

print("完了！")
print(f"Train shape: {X.shape}")
print(f"Test shape: {X_test.shape}")

WCG修正完了
完了！
Train shape: (891, 45)
Test shape: (418, 45)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier

# ==========================================
# 0. データ準備
# ==========================================
df, df_test = df.align(df_test, join='left', axis=1, fill_value=0)
y = df['Perished'].values
X = df.drop('Perished', axis=1).values
X_test = df_test.drop('Perished', axis=1).values

imputer = SimpleImputer(strategy='median')

# ==========================================
# 1. LGBMパート: 徹底的に厳しくする
# ==========================================
pipe_lgbm = Pipeline([
    ('imputer', imputer),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('selection', SelectFromModel(
        LGBMClassifier(n_estimators=100, random_state=0, verbose=-1),
        threshold=-np.inf
    )),
    ('model', LGBMClassifier(random_state=0, verbose=-1))
])

# ★ここを修正: さらに厳しく！
param_grid_lgbm = {
    # 特徴量を10~15個まで絞る（20個だとまだ多い可能性があります）
    'selection__max_features': [10, 15],

    # 木の深さを3に固定（かなり浅くする）
    'model__max_depth': [3],

    # 葉っぱの最小データ数を20以上に増やす（小規模な例外を無視させる）
    'model__min_child_samples': [20, 30],

    # 正則化係数を大きくする
    'model__reg_alpha': [1.0, 5.0],
    'model__reg_lambda': [1.0, 5.0]
}

print("--- LGBM GridSearch ---")
gs_lgbm = GridSearchCV(pipe_lgbm, param_grid_lgbm, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
gs_lgbm.fit(X, y)
best_lgbm = gs_lgbm.best_estimator_
print(f"LGBM Best CV: {gs_lgbm.best_score_:.4f}")


# ==========================================
# 2. RandomForestパート: こちらも厳しく
# ==========================================
pipe_rf = Pipeline([
    ('imputer', imputer),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('selection', SelectFromModel(
        RandomForestClassifier(n_estimators=100, random_state=0),
        threshold=-np.inf
    )),
    ('model', RandomForestClassifier(random_state=0))
])

# ★ここを修正
param_grid_rf = {
    'selection__max_features': [10, 15],
    'model__n_estimators': [100, 200], # 300は多すぎるかも
    'model__max_depth': [4, 5],        # 6からさらに下げる
    'model__min_samples_leaf': [5, 10] # かなり大きなグループしか認めない
}

print("\n--- RandomForest GridSearch ---")
gs_rf = GridSearchCV(pipe_rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
gs_rf.fit(X, y)
best_rf = gs_rf.best_estimator_
print(f"RF Best CV: {gs_rf.best_score_:.4f}")


# ==========================================
# 3. LogisticRegressionパート
# ==========================================
pipe_lr = Pipeline([
    ('imputer', imputer),
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(C=0.5, random_state=0, max_iter=1000)) # Cを小さくして正則化
])

from sklearn.ensemble import StackingClassifier

# ==========================================
# 4. Stacking (Votingの進化版)
# ==========================================
# 最終決定者（Final Estimator）はシンプルなロジスティック回帰が鉄板です
final_layer = LogisticRegression(random_state=0)

stacking_model = StackingClassifier(
    estimators=[
        ('lgbm', best_lgbm),
        ('rf', best_rf),
        ('lr', pipe_lr)
    ],
    final_estimator=final_layer,
    cv=5,  # 内部でCVを行いながら学習するのでリークしません
    n_jobs=-1
)

print("\n--- Stacking Fitting ---")
stacking_model.fit(X, y)

# 評価
print(f"Stacking Train Score: {stacking_model.score(X, y):.4f}")

from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(stacking_model, X, y, cv=5, scoring='accuracy')
print(f"CV平均スコア: {cv_scores.mean():.4f}")



--- LGBM GridSearch ---
Fitting 5 folds for each of 16 candidates, totalling 80 fits
LGBM Best CV: 0.7868

--- RandomForest GridSearch ---
Fitting 5 folds for each of 16 candidates, totalling 80 fits
RF Best CV: 0.8137

--- Stacking Fitting ---
Stacking Train Score: 0.8597
CV平均スコア: 0.8339


In [ ]:
# ==========================================
# 6. Pseudo-Labeling (擬似ラベリング)
# ==========================================
print("\n--- Pseudo-Labeling 開始 ---")

# 1. まず、テストデータの「確率」を出します
test_probs = stacking_model.predict_proba(X_test)[:, 1] # 生存確率(1)を取得

# 2. 「自信満々なデータ」だけを抽出します
# 閾値設定: 92%以上自信があるなら生存(1)、8%以下なら死亡(0)とみなす
confident_mask = (test_probs > 0.92) | (test_probs < 0.08)
confident_indices = np.where(confident_mask)[0]

print(f"自信満々なテストデータ数: {len(confident_indices)}")

# 3. そのデータを学習データに追加します
if len(confident_indices) > 0:
    # 自信のあるX_testを抽出
    X_pseudo = X_test[confident_indices]

    # 自信のある予測ラベル(0 or 1)を作成
    y_pseudo = np.where(test_probs[confident_indices] > 0.5, 1, 0)

    # 元の学習データと合体！
    X_new = np.vstack([X, X_pseudo])
    y_new = np.concatenate([y, y_pseudo])

    print(f"学習データ増強: {len(X)} -> {len(X_new)}")

    # 4. 増えたデータでもう一度学習 (再fit)
    stacking_model.fit(X_new, y_new)
    print("再学習完了！")

# ==========================================
# 7. 最終予測と提出
# ==========================================
final_predictions = stacking_model.predict(X_test)




--- Pseudo-Labeling 開始 ---
自信満々なテストデータ数: 49
学習データ増強: 891 -> 940
再学習完了！


In [ ]:
path = '/content/drive/MyDrive/02.コンペ1（10 28公開済）/'

submission = pd.read_csv(path + 'gender_submission.csv')
submission

,PassengerId,Perished
0,892,1
1,893,0
2,894,1
3,895,1
4,896,0
...,...,...
413,1305,1
414,1306,0
415,1307,1
416,1308,1


In [ ]:
final_predictions.shape

(418,)

In [ ]:
final_predictions

array([1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,

In [ ]:
submission['Perished'] = final_predictions
submission

,PassengerId,Perished
0,892,1
1,893,1
2,894,1
3,895,1
4,896,0
...,...,...
413,1305,1
414,1306,0
415,1307,1
416,1308,1


In [ ]:
# Google Drive・Google Colaboratoryで作業する場合
submission.to_csv('submission.csv',index=False)

from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>